In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score,confusion_matrix, silhouette_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.datasets import make_blobs


import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

In [ ]:
data = pd.read_csv("prj-ML-model-LT_OV30/modeling_data.csv", encoding = "cp949")
print(data.head())
print(data.shape)

In [ ]:
# 건물 정보 없는 데이터 삭제 
filtered_data = data.dropna(subset=['pick_건물용도', 'dlvry_건물용도'])
print(filtered_data.shape) #44,644 
print(filtered_data['outcome'].value_counts()) # 38,816, 5,828 

In [ ]:
print(filtered_data.isna().sum())

In [ ]:
print(filtered_data.dtypes)

In [ ]:
# dlvry_지하층수 음수로
# filtered_data['dlvry_지하층수'] = pd.to_numeric(filtered_data['dlvry_지하층수'], errors='coerce')
# filtered_data['dlvry_지상층수'] = pd.to_numeric(filtered_data['dlvry_지상층수'], errors='coerce')
# filtered_data['pick_floor'] = pd.to_numeric(filtered_data['pick_floor'], errors='coerce')

filtered_data['dlvry_지하층수'] = filtered_data['dlvry_지하층수'].apply(lambda x: -x)
#print(filtered_data.head())
print(filtered_data.isna().sum())

## 1. 데이터 전처리


## 1-1. pick_건물용도/ dlvry_건물용도 : 개수 축소

In [ ]:
print(filtered_data['pick_건물용도'].value_counts())

In [ ]:
# 데이터 정리
values_to_replace = ['위락시설', '문화시설', '종교시설', '체육시설','위험물저장및처리시설', '자동차관련시설']
filtered_data['pick_건물용도'] = data['pick_건물용도'].replace(values_to_replace, '기타시설')
print(filtered_data['pick_건물용도'].value_counts())

In [ ]:
print(filtered_data['dlvry_건물용도'].value_counts())

In [ ]:
value_counts = filtered_data['dlvry_건물용도'].value_counts()

threshold = 100
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].apply(lambda x: '기타시설' if value_counts[x] <= threshold else x)

filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('교육연구시설', '교육시설')
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('공공용시설', '공공시설')

print(filtered_data['dlvry_건물용도'].value_counts())

## 1-2. dlvry_지상층수 /지하층수 : dlvry_건물용도 median 값 가져오기

In [ ]:
median_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].median()
mean_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].mean()

print("Median Values:")
print(median_values)

print("\nMean Values:")
print(mean_values)


In [ ]:
# na 값을 중앙값으로 채우기
filtered_data['dlvry_지상층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지상층수']), inplace=True)
filtered_data['dlvry_지하층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지하층수']), inplace=True)

print(filtered_data.describe())
print(filtered_data.columns)

In [ ]:
print(filtered_data.isna().sum())

## 1-3.  pick_floor, dlvry_지상층수, dlvry_지하층수 

In [ ]:
print(filtered_data['pick_floor'].value_counts())

In [ ]:
print(filtered_data['dlvry_지하층수'].value_counts())

In [ ]:
print(filtered_data['dlvry_지상층수'].value_counts())
print(filtered_data['dlvry_지상층수'].describe())

In [ ]:
def categorize_floor(value):
    if value < 0:
        return '지하'
    elif 0 <= value <= 5:
        return '저층'
    elif 6 <= value <= 10 :
        return '중층'
    elif 11 <= value <= 15 : 
        return '중상층'
    elif 16 <= value <= 29 :
        return '고층'
    elif 30 <= value <= 49 :
        return '준초고층'
    elif 50 <= value:
        return '초고층'

filtered_data['pick_floor'] = filtered_data['pick_floor'].apply(categorize_floor)
filtered_data['dlvry_지상층수'] = filtered_data['dlvry_지상층수'].apply(categorize_floor)

print(filtered_data['dlvry_지상층수'].value_counts())
print(filtered_data['pick_floor'].value_counts())

## 1-4. pick_rgn2_nm, dlvry_rgn2_nm, dlvry_rgn3_nm 정제

In [ ]:
print(filtered_data['dlvry_rgn2_nm'].value_counts())
print(filtered_data['dlvry_rgn3_nm'].value_counts())

## 2. 데이터 전처리

In [ ]:
print(filtered_data.shape) #44,644
print(filtered_data.info())

In [ ]:
dataset = filtered_data[[ 'reg_hour', 'ord_price','actual_dlvry_distance', 'pick_floor', 
                         'pick_category', 'pick_건물용도','dlvry_지상층수', 'dlvry_지하층수',
                         'dlvry_건물용도','day_of_week', 'is_holiday', '기온', 'dlvry_rgn2_nm', 'outcome'
                        ]]
dataset.isna().sum()
dataset.to_csv('prj-ML-model-LT_OV30/modeling_data_final.csv', index= False, encoding = "utf-8")

In [ ]:
print(dataset['outcome'].value_counts())
print(dataset['outcome'].isna().sum())

## 2-1. factor 변수 / one-hot-encoding

In [ ]:
# category  
for col in ['reg_hour','pick_floor',  'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm', 'outcome' ] : 
    dataset[col] = dataset[col].astype('category')

print(dataset.dtypes)

In [ ]:
var = ['reg_hour','pick_floor',  'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm' ] 
encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(dataset[var]).toarray(), columns=encoder.get_feature_names_out(var), index = dataset.index)
dataset = pd.concat([onehot, dataset.drop(columns=var)], axis=1)

print(dataset.columns)

## 3. train/test set split

In [ ]:
outcome_counts = dataset['outcome'].value_counts()
train_set, test_set = train_test_split(dataset, test_size=0.25, stratify = dataset['outcome'], random_state=1234)

train_outcome_counts = train_set['outcome'].value_counts()
test_outcome_counts = test_set['outcome'].value_counts()

print(train_outcome_counts)
print(test_outcome_counts)

print(train_set.shape) #33483
print(test_set.shape) #11161

In [ ]:
X_train = train_set.drop(columns=['outcome'])
y_train = train_set['outcome']

X_test = test_set.drop(columns=['outcome'])
y_test = test_set['outcome']

print(X_train.columns)

In [ ]:
# set.seed(1234)
# train_ratio = 0.75
# total_samples = dataset.shape[0]
# train_samples = int(train_ratio * total_samples)
# train_set = dataset[:train_samples]
# test_set = dataset[train_samples:]

# print(train_set.shape) #33483
# print(test_set.shape) #11161

## 4. numeric variables

In [ ]:
num_vars = ['ord_price', 'actual_dlvry_distance', 'dlvry_지하층수', '기온']
scaler = MinMaxScaler()

X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
X_test[num_vars] = scaler.transform(X_test[num_vars])

#print(X_train.head())
print(X_train.shape) #33483
print(X_test.shape) #11161

## 5. modeling - Logistic Regression

In [ ]:
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

# 다중 공선성 확인
vif_result = calculate_vif(X_train)
print(vif_result)


In [ ]:
glm_1 = sm.GLM(y_train, X_train, family = sm.families.Binomial())
glm1_fit = glm_1.fit()
print(glm1_fit.summary())

joblib.dump(glm1_fit,'prj-ML-model-LT_OV30/glm_model.joblib')

In [ ]:
y_pred = glm1_fit.predict(X_test)

threshold = 0.5 # cut-off 값 
y_pred_binary = (y_pred > threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)

sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

print("\nConfusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy) #0.87
print("Sensitivity (True Positive Rate):", sensitivity) # 0.18
print("Specificity (True Negative Rate):", specificity) # 0.98

test = conf_matrix[1,1] / (conf_matrix[0, 1] + conf_matrix[1, 1])
print(test)

In [ ]:
print(y_pred.shape) #11,161                             

In [ ]:
# logit_model = LogisticRegression(penalty = 'none')
# cv_scores = cross_val_score(logit_model, X_train, y_train, cv=5, scoring='accuracy')
# mean_cv_accuracy = cv_scores.mean()
# print("Cross-Validation Mean Accuracy:", mean_cv_accuracy)

# logit_model.fit(X_train, y_train)
# print("Model Coefficients:", logit_model.coef_)


In [ ]:
# X_train = sm.add_constant(X_train) 
# logit_model = sm.Logit(y_train, X_train)
# result = logit_model.fit()

# print(result.summary())

# X_test = sm.add_constant(X_test) 
# y_pred = result.predict(X_test)  

# y_pred_binary = (y_pred > 0.5).astype(int)

# conf_matrix = confusion_matrix(y_test, y_pred_binary)
# accuracy = accuracy_score(y_test, y_pred_binary)
# sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
# specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# print("\nConfusion Matrix:")
# print(conf_matrix)
# print("\nAccuracy:", accuracy)
# print("Sensitivity (True Positive Rate):", sensitivity)
# print("Specificity (True Negative Rate):", specificity)


In [ ]:
# glm_2 = LogisticRegression(penalty='none')
# glm_2.fit(X_train, y_train)

# variable_names = X_train.columns
# coef = glm_2.coef_[0]

# coef_table = pd.DataFrame({'Variable': variable_names, 'Coefficient': coef})

# logit_model = sm.Logit(y_train, X_train)
# result = logit_model.fit()
# p_values = result.pvalues
# coef_table['p-value'] = p_values

# print(coef_table)
# pred_glm = glm_2.predict(X_test)
# class_report = classification_report(y_test, pred_glm)
# print(class_report)

## 6. clustering

In [ ]:
def visualize_silhouette_scores(data, method='kmeans', param_init='random', param_n_init=10, param_max_iter=300):
    clusters_range = range(2, 15)
    results = []

    for i in clusters_range:
        if method == 'kmeans':
            clusterer = KMeans(n_clusters=i, init=param_init, n_init=param_n_init, max_iter=param_max_iter, random_state=0)
        elif method == 'agglomerative':
            clusterer = AgglomerativeClustering(n_clusters=i)
        else:
            raise ValueError("Invalid method. Choose 'kmeans' or 'agglomerative'.")

        cluster_labels = clusterer.fit_predict(data)
        silhouette_avg = silhouette_score(data, cluster_labels)
        results.append([i, silhouette_avg])

    result = pd.DataFrame(results, columns=["n_clusters", "silhouette_score"])
    
    plt.figure()
    sns.heatmap(pd.pivot_table(result, index="n_clusters", values="silhouette_score"),annot=True, linewidths=.5, fmt='.3f', cmap=sns.cm._rocket_lut)
    plt.tight_layout()
    plt.title(f"Silhouette Scores for {method.capitalize()} Clustering")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.show()


In [ ]:
data  = y_pred.to_numpy().reshape(-1, 1)

visualize_silhouette_scores(data, method='kmeans')
visualize_silhouette_scores(data, method='agglomerative')

In [ ]:
X = y_pred.to_numpy().reshape(-1, 1)
n = 5

kmeans = KMeans(n_clusters=n)
kmeans_clusters = kmeans.fit_predict(X)

agglomerative = AgglomerativeClustering(n_clusters=n)
agglomerative_clusters = agglomerative.fit_predict(X)

kmeans_cluster_centers = kmeans.cluster_centers_
agg_cluster_centers = np.array([X[agglomerative_clusters == i].mean(axis=0) for i in range(n)])

# 데이터 시각화
plt.figure(figsize=(12, 4))

plt.subplot(131)
plt.scatter(X, y_pred, c=y_pred, cmap='viridis')
plt.title('True Clusters')
plt.xlabel('X')
plt.ylabel('Y_pred')

plt.subplot(132)
plt.scatter(X, y_pred, c=kmeans_clusters, cmap='viridis')
#plt.scatter(kmeans_cluster_centers, [y_pred.mean()] * len(kmeans_cluster_centers), c='red', marker='x', s=100, label='Cluster Centers')
#plt.title('K-Means Clusters')
plt.xlabel('X')
plt.ylabel('Y_pred')

plt.subplot(133)
plt.scatter(X, y_pred, c=agglomerative_clusters, cmap='viridis')
#plt.scatter(agg_cluster_centers, [y_pred.mean()] * len(agg_cluster_centers), c='red', marker='x', s=100, label='Cluster Centers')
#plt.title('Agglomerative Clusters')
plt.xlabel('X')
plt.ylabel('Y_pred')

plt.legend()
plt.show()


In [ ]:
kmeans_silhouette_score = silhouette_score(X, kmeans_clusters)
agglomerative_silhouette_score = silhouette_score(X, agglomerative_clusters)

print(f"K-Means Silhouette Score: {kmeans_silhouette_score:.2f}")
print(f"Agglomerative Silhouette Score: {agglomerative_silhouette_score:.2f}")

In [ ]:
test_data = pd.concat([test_set, y_pred, pd.DataFrame(kmeans_clusters, columns=['KMeansCluster']),
                       pd.DataFrame(agglomerative_clusters, columns=['AgglomerativeCluster'])], axis=1)

print(test_data.columns)

## 7. 최근 3일 서초구 배차건 대입

In [ ]:
new_data = pd.read_csv("prj-ML-model-LT_OV30/new_data_final.csv", encoding="utf-8")
print(new_data.head(2))

In [ ]:
print(new_data.isna().sum())
print(new_data.shape) #18213

In [ ]:
new_data[['dlvry_rgn1_nm', 'dlvry_rgn2_nm', 'dlvry_rgn3_nm', 'etc']] = new_data['dlvry_address'].str.split(' ', n=3, expand=True)

# dlvry_지하층수 음수로
new_data['dlvry_지하층수']  = new_data['dlvry_지하층수'].apply(lambda x: -x)
print(new_data.columns)

## 7-1. pick_건물용도/dlvry_건물용도 : 개수 축소 

In [ ]:
print(new_data['pick_건물용도'].value_counts())

In [ ]:
# 데이터 정리
values_to_replace = ['위락시설', '문화시설', '종교시설', '체육시설','위험물저장및처리시설', '자동차관련시설','운수시설']
new_data['pick_건물용도'] = new_data['pick_건물용도'].replace(values_to_replace, '기타시설')
print(new_data['pick_건물용도'].value_counts())

In [ ]:
print(new_data['dlvry_건물용도'].value_counts())

In [ ]:
new_counts = new_data['dlvry_건물용도'].value_counts()

threshold = 100
new_data['dlvry_건물용도'] = new_data['dlvry_건물용도'].apply(lambda x: '기타시설' if new_counts[x] <= threshold else x)

new_data['dlvry_건물용도'] = new_data['dlvry_건물용도'].replace('교육연구시설', '교육시설')
new_data['dlvry_건물용도'] = new_data['dlvry_건물용도'].replace('공공용시설', '공공시설')

print(new_data['dlvry_건물용도'].value_counts())

## 7-3. dlvry_지상층수/지하층수 : dlvry_건물용도별 median 값 가져오기

In [ ]:
median_values =new_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].median()
mean_values = new_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].mean()

#print(median_values)
#print(mean_values)

# na 값을 중앙값으로 채우기
new_data['dlvry_지상층수'].fillna(new_data['dlvry_건물용도'].map(median_values['dlvry_지상층수']), inplace=True)
new_data['dlvry_지하층수'].fillna(new_data['dlvry_건물용도'].map(median_values['dlvry_지하층수']), inplace=True)

#print(new_data.describe())
print(new_data.isna().sum())

In [ ]:
new_data['pick_floor'] = new_data['pick_floor'].apply(categorize_floor)
new_data['dlvry_지상층수'] = new_data['dlvry_지상층수'].apply(categorize_floor)

print(new_data['dlvry_지상층수'].value_counts())
print(new_data['pick_floor'].value_counts())

## 7-4. new_data 전처리

In [ ]:
print(new_data.shape) #18,213
print(new_data.info()) 

In [ ]:
new_model_df = new_data[['reg_hour', 'ord_price','actual_dlvry_distance', 'pick_floor', 
                         'pick_category', 'pick_건물용도','dlvry_지상층수', 'dlvry_지하층수',
                         'dlvry_건물용도','day_of_week', 'is_holiday', '기온', 'dlvry_rgn2_nm'
                        ]]
new_model_df.isna().sum()


## 7-5. factor 변수 encoding

In [ ]:
for col in ['reg_hour','pick_floor',  'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm', 'outcome' ] : 
    new_model_df[col] = new_model_df[col].astype('category')

print(new_model_df.dtypes)

In [ ]:
var = ['reg_hour','pick_floor',  'pick_category', 'pick_건물용도','dlvry_지상층수','dlvry_건물용도',
            'day_of_week', 'is_holiday','dlvry_rgn2_nm' ] 
encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(new_model_df[var]).toarray(), columns=encoder.get_feature_names_out(var), index = new_model_df.index)
new_model_df = pd.concat([onehot, new_model_df.drop(columns=var)], axis=1)

print(new_model_df.columns)

In [ ]:
# numeric scale
new_model_df[num_vars] = scaler.transform(new_model_df[num_vars])
print(new_model_df.head(2))

## 7-6. modeling 적용

In [ ]:
y_pred_new = glm1_fit.predict(new_model_df)
print(y_pred_new.shape)
